In [92]:
import os
os.environ["DEEPLAKE_DOWNLOAD_PATH"] = "./dataset"

In [93]:
import deeplake
from torchvision import transforms
from pprint import pprint
import numpy as np

ds = deeplake.dataset('hub://activeloop/wiki-art', access_method = 'local')

filtered_labels = ['realism', 'expressionism', 'romanticism'] #? Top Three
# filtered_labels = ["action_painting", "analytical_cubism", "synthetic_cubism"] #? Bottom three, for testing
@deeplake.compute
def filter_labels(sample_in, labels_list):
    return sample_in.labels.data()['text'][0] in labels_list

ds_view = ds.filter(filter_labels(filtered_labels), scheduler = 'threaded', num_workers = 0)
first_eight_percent = int(len(ds_view) * 0.8)
train = ds_view[:first_eight_percent]
validate = ds_view[first_eight_percent:]

print("Unfiltered len:", len(ds))
print("Filtered len:", len(ds_view))
print("Train:", len(train))
print("Validate:", len(validate))

transformation = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4982, 0.4496, 0.3861),
                         (0.2846, 0.2787, 0.2712)),
    transforms.CenterCrop(32)
])

wikiart_train = train.pytorch( 
    transform={'images': transformation, 'labels': None},
    tensors=["images", "labels"],
    # decode_method={'images':'pil', 'labels': 'numpy'},
    shuffle=True,
    batch_size=32
)

wikiart_validate = validate.pytorch( 
    transform={'images': transformation, 'labels': None},
    tensors=["images", "labels"],
    # decode_method={'images':'pil', 'labels': 'numpy'},
    shuffle=True,
    batch_size=32
)

./dataset\hub_activeloop_wiki-art loaded successfully.
** Loaded local copy of dataset. Downloaded on: Wed Dec 14 00:48:59 2022


100%|██████████| 81433/81433 [00:02<00:00, 28291.77it/s]
c:\Users\ramon\Developer\.venv\lib\site-packages\deeplake\integrations\pytorch\common.py:91: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


Unfiltered len: 81433
Filtered len: 24484
Train: 19587
Validate: 4897


In [94]:
import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.conv1 = nn.Conv2d(3,n,kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n,n//2,kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(n//2,n//2,kernel_size=3, padding=1)

        self.ch = 4 * 4 * (n//2)
        self.fc1 = nn.Linear(self.ch, 32)
        self.fc2 = nn.Linear(32, 27)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = F.max_pool2d(torch.tanh(self.conv3(out)), 2)
        out = out.view(-1, self.ch)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out

class Net_Skip(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.conv1 = nn.Conv2d(3,n,kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n,n//2,kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(n//2,n//2,kernel_size=3, padding=1)

        self.ch = 4 * 4 * (n//2)
        self.fc1 = nn.Linear(self.ch, 32)
        self.fc2 = nn.Linear(32, 3)
    
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out1 = out
        out = F.max_pool2d(torch.tanh(self.conv3(out)) + out1, 2)
        out = out.view(-1, self.ch)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out


class Net_Dropout(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.conv1 = nn.Conv2d(3,n,kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout2d(p=0.3)
        self.conv2 = nn.Conv2d(n,n//2,kernel_size=3, padding=1)
        self.conv2_dropout = nn.Dropout2d(p=0.3)
        self.conv3 = nn.Conv2d(n//2,n//2,kernel_size=3, padding=1)
        self.conv3_dropout = nn.Dropout2d(p=0.3)

        self.ch = 4 * 4 * (n//2)
        self.fc1 = nn.Linear(self.ch, 32)
        self.fc2 = nn.Linear(32, 3)
    
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = self.conv1_dropout(out)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = self.conv2_dropout(out)
        out = F.max_pool2d(torch.tanh(self.conv3(out)), 2)
        out = self.conv3_dropout(out)
        out = out.view(-1, self.ch)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out

def training_function(train_loader, model, loss_fn, optimizer, device, n_epochs):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            labels = labels.flatten() # this is needed because for some reason every label is in a seperate tensor
            # print(labels)
            # print(imgs.shape)
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs).to(device=device)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        print('{} Epoch {}, Training loss {}'.format(
            datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))


def validate(model, train_loader, val_loader, device):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        with torch.no_grad():
            for imgs, labels in loader:
                # labels = labels.flatten()
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs).to(device=device)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted == labels).sum())
        print("Accuracy {}: {:.6f}".format(name, correct / total))

In [95]:
import torch
import torch.optim as optim

device = (torch.device('cuda') if torch.cuda.is_available()
                      else torch.device('cpu'))

network = Net(32)

print(f"Training on {device}:")
print(f"\tdevice: {torch.cuda.get_device_name(0)}")
print("\tSize of wikiart:", len(wikiart_train))

training_function(
    train_loader = wikiart_train,
    model = network.to(device=device),
    loss_fn = nn.NLLLoss(),
    optimizer = optim.SGD(network.parameters(), lr=1e-2),
    device=device,
    n_epochs = 100,
)

Training on cuda:
	device: NVIDIA GeForce RTX 3060 Ti
	Size of wikiart: 613


Please wait, filling up the shuffle buffer with samples.: 100%|█████████▉| 1.90G/1.91G [00:09<00:00, 213MB/s]


Shuffle buffer filling is complete.
2022-12-16 23:45:25.705236 Epoch 1, Training loss 1.200792768456613


Please wait, filling up the shuffle buffer with samples.: 100%|█████████▉| 1.91G/1.91G [00:09<00:00, 226MB/s]


Shuffle buffer filling is complete.
2022-12-16 23:49:55.222351 Epoch 2, Training loss 0.9207099290111909


...

Shuffle buffer filling is complete.
2022-12-17 06:46:32.434643 Epoch 98, Training loss 0.8021115632469463


Please wait, filling up the shuffle buffer with samples.: 100%|█████████▉| 1.90G/1.91G [00:08<00:00, 230MB/s]


Shuffle buffer filling is complete.
2022-12-17 06:50:53.060266 Epoch 99, Training loss 0.802694385631244


Please wait, filling up the shuffle buffer with samples.: 100%|█████████▉| 1.91G/1.91G [00:09<00:00, 213MB/s]


Shuffle buffer filling is complete.
2022-12-17 06:55:12.952489 Epoch 100, Training loss 0.8104986553771655


In [96]:
validate(network.to(device=device), wikiart_train, wikiart_validate, device)

Please wait, filling up the shuffle buffer with samples.: 100%|█████████▉| 1.90G/1.91G [00:08<00:00, 237MB/s]


Shuffle buffer filling is complete.
Accuracy train: 15.177567


Please wait, filling up the shuffle buffer with samples.: 100%|█████████▉| 1.91G/1.91G [00:09<00:00, 221MB/s]


Shuffle buffer filling is complete.
Accuracy val: 0.006535


In [97]:
torch.save(network.state_dict(), './models/net_version_two.pt') #! REMEMEBER TO NOT RUN AGAIN WITHOUT COMMENTING OUT FUCKER 